Importing

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
# from gensim.summarization import summarize
from sklearn.metrics import mean_squared_error
from math import sqrt

ModuleNotFoundError: No module named 'gensim.summarization'

Adding commentary data

In [12]:
# from google.colab import files
# uploaded = files.upload()

In [13]:
# Load the csv file
df = pd.read_csv("database\IPL_Match_Highlights_Commentary.csv")

Preprocessing of data.

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Harsh
[nltk_data]     Upadhayay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\Harsh
[nltk_data]     Upadhayay\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
# Convert all text to lowercase
df["comment"] = df["comment"].astype(str).str.lower()

# Remove punctuation
df["comment"] = df["comment"].astype(str).str.replace('[^\w\s]','')

# Remove stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df["comment"] = df["comment"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Tokenize words
df["comment"] = df["comment"].apply(lambda x: word_tokenize(x))

# Lemmatize words
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df["comment"] = df["comment"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Combine the tokenized words into a single string
df["comment"] = df["comment"].apply(lambda x: " ".join(x))

# Create a list of all the preprocessed comments
preprocessed_comments = df["comment"].tolist()

preprocessed_comments

C:\Users\Harsh Upadhayay\AppData\Local\Temp\ipykernel_6712\2662994380.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df["comment"] = df["comment"].astype(str).str.replace('[^\w\s]','')
[nltk_data] Downloading package stopwords to C:\Users\Harsh
[nltk_data]     Upadhayay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Harsh
[nltk_data]     Upadhayay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['nehra mandeep four first boundary mandeep rcb full pad needed put away mandeep picked dispatched midwicket couple bounce fence',
 'nehra mandeep four backtoback boundary end first nehra tad short length mandeep width cut didnt try hit hard placed behind point bhuvi third man gave chase pretty quickly',
 'henriques kedar jadhav four hit straight back henriques late get hand offcutter almost fooled jadhav shimmied checked drive middled alright beat midoff fielder',
 'nehra kedar jadhav four another full toss jadhav time pick spot deep midwicket fence great bowling nehra missing yorker quite bit',
 'nehra kedar jadhav four four jadhav starting really find timing looking dangerous smart batting rather throwing bat everything know fine leg wait back length delivery come pulling fielder nehra pressure',
 'henriques kedar jadhav kedar jadhav run 1 run completed cutting conjured wicket nowhere thats sensational throw deep goodness absolutely nailed stump long leg jadhav huge moment context m

In [18]:
# Split the preprocessed comments into training and testing sets
split_ratio = 0.8
train_size = int(len(preprocessed_comments) * split_ratio)
train_comments = preprocessed_comments[:train_size]
test_comments = preprocessed_comments[train_size:]

# Convert the preprocessed comments into a numerical format
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(train_comments)
test_vectors = vectorizer.transform(test_comments)


In [20]:
# Train the Word2Vec model on the preprocessed comments
model = gensim.models.Word2Vec(preprocessed_comments, window=5, min_count=5, workers=4)


In [22]:
sentences = [['southee', 'bowls', 'a', 'perfect', 'delivery'], ['batsman', 'hits', 'southee', 'for', 'a', 'six']]
model = gensim.models.Word2Vec(sentences, min_count=1)
sentences = [['gayle', 'bowls', 'a', 'underway', 'delivery'], ['batsman', 'hits', 'southee', 'for', 'a', 'four']]
model = gensim.models.Word2Vec(sentences, min_count=1)

Adding player names to model vocabulary.

In [9]:
# from google.colab import files
# uploaded = files.upload()

Adding player names to model vocabulary.

In [23]:
import csv

In [24]:
player_names = []
# Read player names from csv file
with open('database/names.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        player_names.append(row[0])

In [25]:
# Add player names to vocabulary
model.build_vocab([player_names], update=True)

Training the model on text8 corpus.

In [26]:
import gensim.downloader as api

In [27]:
# Download the text8 corpus
corpus = api.load('text8')

[==================================================] 100.0% 31.6/31.6MB downloaded


In [28]:
# Add the corpus to the model's vocabulary
model.build_vocab(corpus, update=True)

In [29]:
# Train the model on the corpus
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

(64094930, 85026035)

Training the model on wikipedia corpus.

In [30]:
import logging
import wikipedia
from gensim.corpora import WikiCorpus
from gensim.utils import simple_preprocess
import requests
import bz2

In [31]:
# Set up logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


this code downloads a large file (over 16 GB in size) from the internet, so it may take several hours to complete depending on the above-mentioned factors. It is important to note that downloading large files can consume a significant amount of bandwidth and can affect the internet speed for other users on the same network.


In [2]:
# Download a Wikipedia dump file in XML format
import requests
wiki_dump_file = 'enwiki-latest-pages-articles.xml.bz2'
url = 'https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2'
r = requests.get(url, stream=True)
with open(wiki_dump_file, 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

In [ ]:
# Use WikiCorpus to extract text from the Wikipedia dump
wiki_corpus = WikiCorpus(wiki_dump_file, dictionary={})
sentences = list(wiki_corpus.get_texts())

In [ ]:
# Preprocess the text data
preprocessed_sentences = [simple_preprocess(sentence) for sentence in sentences]


In [ ]:
# Train a Word2Vec model using the preprocessed text data
model = gensim.models.Word2Vec(preprocessed_sentences, min_count=5, workers=4)

Function to generate summary.

In [ ]:
# Define a function to generate summaries for a given match
def generate_summary(match_id):
    # Get all the preprocessed comments for the given match
    comments = df.loc[df["Match_id"] == match_id, "comment"].tolist()
    
    # Convert the preprocessed comments to vectors using the Word2Vec model
    comment_vectors = [model.wv[comment.split()] for comment in comments]
    
    # Summarize the comments using the TextRank algorithm
    summary = summarize(" ".join(comments), ratio=0.2)
    
    return summary

Generate summary for each match.

In [ ]:
# Generate summaries for all matches in the test set
predicted_summaries = []
for Match_id in df.loc[train_size:, "Match_id"].unique():
    summary = generate_summary(Match_id)
    predicted_summaries.append(summary)

print(predicted_summaries)


Performance measurement.

In [ ]:
# Evaluate the model using mean squared error
actual_summaries = df.loc[train_size:, "comment"].tolist()
mse = mean_squared_error(actual_summaries, predicted_summaries)
rmse = sqrt(mse)
print("Root Mean Squared Error:", rmse)